# TGDS Model with Voltage Delta Prediction

In [ ]:
import context
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import src.data.data_preprocessing as util
import src.models.lstm_model as lstm

tf.compat.v1.set_random_seed(1)

### Set Hyperparameters

In [ ]:
HYPER_PARAMS =  np.load('../../../models/TGDS/training_setup/hyperparameters.npy', allow_pickle=True)
HYPER_PARAMS = HYPER_PARAMS.item()

# add aditional model-spefic hyperparameters
model_hyperparameters = {
    'n_features': 3,                  # number of input features
    'd_t': 0.25,                      # current integration factor
    'loss_funcs': ['mse'],            # loss functions
    'lambda_mse': 1,                  # mse weighting factor
    'lambda_apx': 0,                  # approximation loss weighting factor
    'lambda_mon': 0,                  # monotonicity loss weighting factor   
    'num_repeat': 1,                  # repetition factor for each training profile to create more auxiliary data
    'feature_range_delta_low': 0,     # lower bound of voltage label after scaling\n",
    'feature_range_delta_high': 1,    # upper -||-\n",
    'boundary_delta_low': 0,          # lower bound of voltage value range in V
    'boundary_delta_high': 0.1,       # upper -||-
}

# update hyperparameters
HYPER_PARAMS.update(model_hyperparameters)
HYPER_PARAMS.update({'n_epochs':7})

### Prepare Training/Validation/Test Data

In [ ]:
TRAINING_SETS =  np.load('../../../models/TGDS/training_setup/training_sets.npy', allow_pickle=True)
TRAINING_SETS = TRAINING_SETS.item()

# select correct training set
# training_sets = TRAINING_SETS['10A_all'] + TRAINING_SETS['-10A_all'] + TRAINING_SETS['25A_all'] + TRAINING_SETS['-25A_all']
training_set = TRAINING_SETS['10A_all']

# select arbitrary profile for testing
test_profile = np.random.choice(training_set, 1) 

# repeat profiles to increase training data
train_data = []
for set in training_set:
    set_repeat = [set] * HYPER_PARAMS['num_repeat']
    train_data += set_repeat
    
train_data = np.array(train_data)

# select first profile for validation
validation_profile = [train_data[0]]

# prepare input data
X_train, y_train, scalers_train = util.prepare_current_charge_delta_input(HYPER_PARAMS, train_data, HYPER_PARAMS['stack'], HYPER_PARAMS['cell'])
X_validation, y_validation, _ = util.prepare_current_charge_delta_input(HYPER_PARAMS, validation_profile, HYPER_PARAMS['stack'], HYPER_PARAMS['cell'])
X_test, y_test, _ = util.prepare_current_charge_delta_input(HYPER_PARAMS, test_profile, HYPER_PARAMS['stack'], HYPER_PARAMS['cell'])

### Initialize and Train Model

In [ ]:
lstm = lstm.Model()
lstm.initialize(HYPER_PARAMS)

In [ ]:
_, fig = lstm.train(X_train, y_train, scalers_train)

# save model and hyperparameters
MODEL_ID = str(np.random.randint(10000))

# lstm.model.save('../../../models/TGDS/' + str(MODEL_ID))
# np.save('../../../models/TGDS/' + str(MODEL_ID) + '/hyperparameters', HYPER_PARAMS)
# fig.savefig('../../../reports/figures/theory_guided_charge-' + str(MODEL_ID) + '-learning_curve.png')

### Test Model

In [ ]:
print('Validation Profile:', validation_profile)
print('Test Profile:', test_profile)

yhat_train_unscaled, _, _, _, _, fig = lstm.test(X_train, y_train, X_validation, y_validation, X_test, y_test, scalers_train)

# save plots and predicted sequences
# np.save('../../../models/TGDS/' + str(MODEL_ID) + '/predictions', yhat_train_unscaled)
# fig.savefig('../../../reports/figures/theory_guided_charge-' + str(MODEL_ID) + '-validation&test_profiles.png')

### Prepare Data for Use Cases

In [ ]:
TEST_SETS =  np.load('../../../models/TGDS/training_setup/test_sets.npy', allow_pickle=True)
TEST_SETS = TEST_SETS.item()

test_profiles_usecase_1 = TEST_SETS['Untitled_1']
test_profiles_usecase_2 = TEST_SETS['Untitled_1']
test_profiles_usecase_3 = TEST_SETS['Untitled_1']

X_case_1, y_case_1, _ = util.prepare_current_charge_delta_input(HYPER_PARAMS, test_profiles_usecase_1, HYPER_PARAMS['stack'], HYPER_PARAMS['cell'])
X_case_2, y_case_2, _ = util.prepare_current_charge_delta_input(HYPER_PARAMS, test_profiles_usecase_2, HYPER_PARAMS['stack'], HYPER_PARAMS['cell'])
X_case_3, y_case_3, _ = util.prepare_current_charge_delta_input(HYPER_PARAMS, test_profiles_usecase_3, HYPER_PARAMS['stack'], HYPER_PARAMS['cell'])

### Test Model on Use Cases

In [ ]:
print('Use Case 1:', test_profiles_usecase_1)
print('Use Case 2:', test_profiles_usecase_2)
print('Use Case 3:', test_profiles_usecase_3)

lstm.test_usecases(X_train, y_train, X_case_1, y_case_1, X_case_2, y_case_2, X_case_3, y_case_3, scalers_train)